In [1]:
using LinearAlgebra
using ShadyPolytopes
using Polymake
using Combinatorics

polymake version 4.13
Copyright (c) 1997-2024
Ewgenij Gawrilow, Michael Joswig, and the polymake team
Technische Universität Berlin, Germany
https://polymake.org

This is free software licensed under GPL; see the source for copying conditions.
There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
"""
	is_intersected(signature_vector, facet)

Checks if the relative interior of the facet (given by an index vector)
intersects a linear subspace given by its signature vector.
"""
function is_intersected(signature_vector, facet)
    signs = signature_vector[facet]
    return minimum(signs)<0 && maximum(signs)>0
end

is_intersected

In [3]:
"""
	function intersection_vector(sv, facets)

Takes a vector of all possible signature vectors
for linear subspaces
and a vector of factes (represented by an index vector)

Returns a vector of vectors containing the following:
For each signature vector we calculate which
facets are intersected in their interior.
"""
function intersection_vector(sv, facets)
    [
		[is_intersected(signature_vector,facet) 
		 	for facet in facets] 
		for signature_vector in sv
	]
end

intersection_vector

In [4]:
"""
	ray_signature(ray, hyperplanes)

Takes a `ray` and a vector of hyperplane normals 
and calculates for each hyperplane, which side the ray is on.

"""
function ray_signature(ray, hyperplanes)
    [sign(dot(h, ray)) for h in eachrow(hyperplanes)]
end

ray_signature

In [5]:
"""
	signature_vector(W, V)

Takes a vector of vectors W,
and a vector of vectors V,
then determines all possible 
positions of planes relative to W,
and for each such position calculates the
position of the vertices in V with respect to
this 
Returns a vector of {-1,0,1} vectors.
"""
function signature_vector(W, V)
	hyperplanes = hcat(W...)'
    all_vertices= hcat(V...)'
	R = Vector{Int64}[]
    # hyperplanes is an array whose rows are the hyperplane normals
    HA = Polymake.fan.HyperplaneArrangement(HYPERPLANES = hyperplanes)
    CD = HA.CHAMBER_DECOMPOSITION
    RAYS = eachrow(convert(Array{Rational{Int64}},CD.RAYS))
    for cone_array in CD.CONES
        for row in eachrow(cone_array)
            ray = sum(RAYS[findall(row)])
            push!(R,ray_signature(ray,all_vertices))
        end
    end
    return R
end

signature_vector

In [6]:
Vprime = optimal_icosahedron.positive_vertices
V = all_vertices(optimal_icosahedron)
sig_vec = signature_vector(Vprime,V)

122-element Vector{Vector{Int64}}:
 [1, 0, 1, 0, -1, -1, -1, 0, -1, 0, 1, 1]
 [0, 0, 1, 1, -1, -1, 0, 0, -1, -1, 1, 1]
 [1, 1, 0, 0, -1, -1, -1, -1, 0, 0, 1, 1]
 [0, 1, 1, 1, -1, 0, 0, -1, -1, -1, 1, 0]
 [1, 1, 0, 1, -1, 0, -1, -1, 0, -1, 1, 0]
 [1, 1, -1, 0, -1, 0, -1, -1, 1, 0, 1, 0]
 [0, 1, 1, 1, 0, 1, 0, -1, -1, -1, 0, -1]
 [1, 1, 0, 1, 0, 1, -1, -1, 0, -1, 0, -1]
 [1, 1, -1, 0, 0, 1, -1, -1, 1, 0, 0, -1]
 [1, 0, 0, -1, -1, -1, -1, 0, 0, 1, 1, 1]
 [1, 0, -1, -1, 0, -1, -1, 0, 1, 1, 0, 1]
 [1, 1, -1, -1, 0, 0, -1, -1, 1, 1, 0, 0]
 [-1, 0, 1, 1, -1, 0, 1, 0, -1, -1, 1, 0]
 ⋮
 [-1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1]
 [-1, -1, -1, -1, 1, -1, 1, 1, 1, 1, -1, 1]
 [1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, -1]
 [-1, 1, -1, -1, 1, 1, 1, -1, 1, 1, -1, -1]
 [1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1]
 [1, -1, -1, -1, 1, -1, -1, 1, 1, 1, -1, 1]
 [-1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1]
 [1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1]
 [1, 1, -1, -1, 1, -1, -1, -1, 1, 1, -1, 1]
 [1, 1, 1, 1, 1, 1, 

In [7]:
n = size(V)
# polymake expects a 4xn matrix where each row contains 
# homogeneous coordinates of a vertex
VM_hom = hcat(ones(eltype(V[1]), n), collect(hcat(V...)'))
	
OI_polymake = Polymake.polytope.Polytope(VERTICES = VM_hom)
facets = [findall(row) for row in eachrow(OI_polymake.VERTICES_IN_FACETS)]
inters_vec = intersection_vector(sig_vec, facets)

122-element Vector{Vector{Bool}}:
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]
 [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]
 [1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]
 [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
 [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1]
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0]
 [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
 [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0]
 ⋮
 [1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0]
 [0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
 [1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0

# Minimal number of intersections between a linear subspace and the relative interior of facets

In [8]:
minimum(count.(inters_vec))

4

# Check that the normals are in general position

In [9]:
normals = [h for h in optimal_icosahedron.normals if dot(h,[1,1,1])>0]
ranks = [rank(hcat(subset...)) for subset in combinations(normals, 3)]
all(ranks.==3)

true